# Классификация с помощью дерева решений
## Елисеев Е.В.

# Задание
1. Разработайте программу, которая выполняет классификацию заданного набора данных <br>
с помощью дерева решений. Параметрами программы являются набор данных, критерий выбора <br>
атрибута разбиения (Information gain, Gain ratio, Gini index).
2. Проведите эксперименты на наборе Census Income (данные о результатах переписи населения, <br>
в т.ч. о годовом доходе -- ниже или выше $50000: скачать обучающую выборку в формате CSV, <br>
тестовую выборку в формате CSV, скачать описание). В качестве обучающей выборки для <br>
построения дерева используйте 100% исходных данных.
3. Выполните визуализацию построенных деревьев решений.
4. Доработайте программу, добавив в список ее параметров долю, которую занимает <br>
обучающая выборка от общего размера набора данных, и обеспечив вычисление и выдачу <br>
в качестве результатов следующих показателей качества классификации: аккуратность (accuracy),<br>
 точность (precision), полнота (recall), F-мера.
5. Проведите эксперименты на наборе данных, фиксируя критерий выбора атрибута разбиения<br>
 и варьируя соотношение мощностей обучающей и тестовой выборок от 60%:40% до 90%:10% с шагом 10%.
6. Выполните визуализацию полученных результатов в виде следующих диаграмм:
- построенные деревья решений для заданного набора данных;
- показатели качества классификации в зависимости от соотношения мощностей обучающей <br>
и тестовой выборок для заданного набора данных.
7. Подготовьте отчет о выполнении задания и загрузите отчет в формате PDF в систему. <br>
Отчет должен представлять собой связный и структурированный документ со следующими разделами:<br>
- формулировка задания; 
- гиперссылка на каталог репозитория с исходными текстами, наборами данных и др.
- сопутствующими материалами; 
- рисунки с результатами визуализации; 
- пояснения, раскрывающие смысл полученных результатов.


In [1]:
import sklearn
import pandas as pd
import numpy as np 


train = pd.read_csv('adult.data.csv', header=None)
test = pd.read_csv('adult.test.csv', header=None)
train.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
train.shape

(32561, 15)

In [3]:
column_names = {
    0: "age",
    1: "workclass",
    2: "fnlwgt",
    3: "education",
    4: "education-num",
    5: "marital-status",
    6: "occupation",
    7: "relationship",
    8: "race",
    9: "sex",
    10: "capital-gain",
    11: "capital-loss",
    12: "hours-per-week",
    13: "native-country",
    14: "income"
}

train.rename(columns=column_names, inplace=True)
test.rename(columns=column_names, inplace=True)
train.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
train['income']

0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
32556     <=50K
32557      >50K
32558     <=50K
32559     <=50K
32560      >50K
Name: income, Length: 32561, dtype: object

In [10]:
test.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [5]:
# Проверка на пустые значения
train.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [6]:
# Число данных первого и второго класса
len(train[train['income'] == ' <=50K']), len(train[train['income'] == ' >50K'])

(24720, 7841)

In [7]:
# Применение 0versampling для балансировки недостающего класса
over_50k = train[train['income'] == ' >50K']
over_50k_duplicated = over_50k.sample(frac=2.0, replace=True, random_state=42)

# Объединение исходного DataFrame с дублированными строками
train_augmented = pd.concat([train, over_50k_duplicated])
train_augmented = train_augmented.sample(frac=1, random_state=42)
train_augmented.reset_index(drop=True, inplace=True)

len(train_augmented[train_augmented['income'] == ' <=50K']), len(train_augmented[train_augmented['income'] == ' >50K'])

(24720, 23523)

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


train_x = train.drop(columns='income')
train_y = train['income']
test_x = test.drop(columns='income')
test_y = test['income']

model = DecisionTreeClassifier()
model.fit(train_x, train_y)
y_pred = model.predict(test_x)
